In [2]:
# get stopwords 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# prepare NLTK stopwards
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/amos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# import libraries

import re
import numpy as np
import pandas as pd
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spaCy for preprocessing
#pip3 install spacy
# pip install spacy
import spacy



In [4]:
# remove puntuations
def sent_to_words(sentences: list[str]):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    

# Define function for stopwords
def remove_stopwords(lyrics: list[str]):
  return [
    [ 
      word for word in simple_preprocess(str(doc)) if word not in stop_words
    ] 
    for doc in lyrics
  ]

In [7]:
start_year = 2017

# load dataset 2017
path = os.path.join("..", "datasets", "CompileData", str(start_year) + "_spotify_data_and_lyrics.csv")
df: pd.DataFrame = pd.read_csv(path, header=0)

# remove new line from lyrics
# get first 50 lyrics 
lyrics_list: list[str] = df.lyrics.values.tolist()[:50] 

# remove new line characters
lyrics_list = [re.sub('\\n', ' ', str(song_lyric).lower()) for song_lyric in lyrics_list]
# remove punctuations
lyrics_words = list(sent_to_words(lyrics_list))

# remove stopwords

data_words_nostops = remove_stopwords(lyrics_words)

# print(lyrics_words[0])
print(data_words_nostops[0])

['club', 'best', 'place', 'find', 'lover', 'bar', 'go', 'friends', 'table', 'shots', 'drinking', 'fast', 'talk', 'slow', 'come', 'start', 'conversation', 'trust', 'give', 'chance', 'take', 'hand', 'stop', 'put', 'van', 'man', 'jukebox', 'start', 'dance', 'singing', 'like', 'girl', 'know', 'want', 'love', 'love', 'handmade', 'somebody', 'like', 'come', 'follow', 'lead', 'may', 'crazy', 'mind', 'say', 'boy', 'let', 'talk', 'much', 'grab', 'waist', 'put', 'body', 'come', 'follow', 'lead', 'come', 'come', 'follow', 'lead', 'love', 'shape', 'push', 'pull', 'like', 'magnet', 'although', 'heart', 'falling', 'love', 'body', 'last', 'night', 'room', 'bed', 'sheets', 'smell', 'like', 'every', 'day', 'discovering', 'something', 'brand', 'new', 'love', 'body', 'oh', 'oh', 'oh', 'oh', 'love', 'body', 'oh', 'oh', 'oh', 'oh', 'love', 'body', 'oh', 'oh', 'oh', 'oh', 'love', 'body', 'every', 'day', 'discovering', 'something', 'brand', 'new', 'love', 'shape', 'one', 'week', 'let', 'story', 'begin', 'goi